MULTIMODAL RAG

In [1]:
# Tesseract : PDF 파일에서 문자열 추출하는 OCR(광학문자인식) 라이브러리
# Tesseract 설치
# 윈도우의 경우 깃헙
# Tesseract : (https://github.com/UB-Mannheim/tesseract/wiki)
# Poppler : (https://github.com/oschwartz10612/poppler-windows/releases)
# Poppler는 설치파일 다운로드 후 폴더 경로(... /Library/bin/) 환경변수 추가 필요

# - 참고 : https://tesseract-ocr.github.io/tessdoc/Installation.html

# !sudo apt install tesseract-ocr
# !sudo apt install libtesseract-dev
# !sudo apt-get install poppler-utils

# ! pip install -U langchain openai chromadb langchain-experimental langchain_openai nltk pydantic lxml matplotlib chromadb tiktoken
# ! pip install pillow==11.1.0
# ! pip install "unstructured[all-docs]"==0.17.2

In [ ]:
# 파일 경로
fpath = './data/'
fname = "sample.pdf"

In [3]:
import nltk

nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\wind9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\wind9\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
from unstructured.partition.pdf import partition_pdf
import os

# PDF에서 요소 추출
raw_pdf_elements = partition_pdf(
    filename=os.path.join(fpath, fname),            # PDF 파일 경로
    extract_images_in_pdf=True,                     # 이미지 추출 여부
    extract_image_block_types=["Image", "Table"],   # 추출할 이미지 블록 유형
    chunking_strategy="by_title",                   # 청킹 전략   
    extract_image_block_output_dir=fpath,           # 추출된 이미지 저장 경로
)


In [5]:
# 텍스트, 테이블 추출
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))  # 테이블 요소 추가
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))  # 텍스트 요소 추가

In [6]:
tables[0]

'전년 대비(42주) 2023년 구분 2023년 증감 2024년 (1. 1.∼12. 31.) (%) (1. 1.∼10. 19.) (1. 1.∼10. 21.) 630 (100.0) 673 (100) 전체 663 (100.0) △5.0% 남자 성별 569 (84.5) 526 (83.5) 560 (84.5) △6.1% 1.0% 104 (15.5) 여자 103 (15.5) 104 (16.5) △60.0% 2 ( 0.3) 5 ( 0.7) 5 ( 0.8) 연령 0-9세 20 ( 3.2) 31 ( 4.6) 30 ( 4.5) 10-19세 △33.3% 20-29세 200 (30.2) 209 (33.2) 4.5% 201 (29.9) 30-39세 △18.2% 110 (16.6) 111 (16.5) 90 (14.3) △7.7% 104 (15.7) 96 (15.2) 107 (15.9) 40-49세 118 (17.5) 115 (17.3) △13.9% 50-59세 99 (15.7) 73 (11.6) 60-69세'

In [7]:
texts[0]

'42주차 (10.13.~10.19.)\n\n| Suyztay | aygises zt | Bail\n\n주차\n\n국내발생 해외유입\n\n전체'

텍스트 및 테이블 요약

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 설정
prompt_text = """당신은 표와 텍스트를 요약하여 검색할 수 있도록 돕는 역할을 맡은 어시스턴트입니다.
이 요약은 임베딩되어 원본 텍스트나 표 요소를 검색하는 데 사용될 것입니다.
표 또는 텍스트에 대한 간결한 요약을 제공하여 검색에 최적화된 형태로 만들어 주세요. 표 또는 텍스트: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# 텍스트 요약 체인
model = ChatOpenAI(
    model="qwen2.5-3b-instruct-q4_k_m.gguf",
    base_url="http://localhost:8002/v1",
    api_key="EMPTY",   # 실제로 사용되지 않음
    temperature=0.2,
)
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()
# {"element": lambda x: x} : 입력값을 element 키로 매핑


# 제공된 텍스트에 대해 요약을 할 경우
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})       # max_concurrency : 동시 처리 수
# summarize_chain.batch : 여러 입력값에 대해 비동기적으로 요약 수행


# 요약을 원치 않을 경우
# text_summaries = texts

# 제공된 테이블에 적용
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [10]:
table_summaries[0]

'2023년 대비 2024년까지의 증감률을 요약하면 다음과 같습니다:\n\n- 전체: △5.0%\n- 남자 성별: △6.1%\n- 여자: △60.0%\n- 연령별:\n  - 0-9세: △33.3%\n  - 10-19세: △18.2%\n  - 20-29세: △7.7%\n  - 30-39세: △13.9%\n  - 40-49세: △15.7%\n  - 50-59세: △15.2%\n  - 60-69세: △15.9%\n\n이 요약은 원본 데이터의 주요 특징을 간략하게 요약하고, 검색에 최적화된 형태로 제공되었습니다.'

In [11]:
text_summaries[0]

'42주차 (10월 13일 ~ 10월 19일) 요약: 국내에서 발생한 코로나19 환자 수와 해외에서 유입된 환자 수를 요약한 데이터입니다. 표에는 "국내발생 해외유입" 항목이 전반적인 코로나19 발생 현황을 나타내고 있습니다.'

요약에 약 16분 가량(GTX 1060 6GB 기준) 걸렸지만, 더 알아보기 쉬운 내용으로 요약된 것을 확인할 수 있다.

이미지 요약

In [12]:
import base64


def encode_image(image_path) -> str:
    # 이미지 base64 인코딩
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


# 이미지의 base64 인코딩을 저장하는 리스트
img_base64_list = []

# 이미지를 읽어 base64 인코딩 후 저장
for img_file in sorted(os.listdir(fpath)):              # sorted : 파일 이름 순서대로 정렬
    if img_file.endswith('.jpg'):                       # endswith : 특정 확장자 파일만 선택
        img_path = os.path.join(fpath, img_file)        #  이미지 파일 경로
        base64_image = encode_image(img_path)           # 이미지 base64 인코딩 : base64로 인코딩하는 이유는 텍스트 기반 시스템에서 이미지를 쉽게 전송하고 저장하기 위함
        img_base64_list.append(base64_image)            # 인코딩된 이미지 저장

In [13]:
len(img_base64_list)

49

In [21]:
# 이미지 리사이즈 및 재인코딩 함수

import base64
import io
from PIL import Image


def resize_image_base64(
    img_base64: str,
    max_size: int = 512,
    quality: int = 85,
) -> str:
    """
    base64 이미지를 받아서
    - 최대 변을 max_size로 축소
    - JPEG로 재인코딩
    - base64 문자열 반환
    """
    img_bytes = base64.b64decode(img_base64)
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

    # 비율 유지하면서 축소
    img.thumbnail((max_size, max_size))

    buf = io.BytesIO()
    img.save(buf, format="JPEG", quality=quality, optimize=True)

    return base64.b64encode(buf.getvalue()).decode("utf-8")


In [22]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI


def image_summarize(img_base64: str) -> str:
    # 이미지 리사이즈 및 재인코딩
    img_base64 = resize_image_base64(img_base64, max_size=512)
    
    # 이미지 요약
    chat = ChatOpenAI(
    model="qwen2.5-vl-3b-instruct-q5_k_m.gguf",
    base_url="http://localhost:8003/v1",
    api_key="EMPTY",   # 실제로 사용되지 않음
    temperature=0.2,
    )
    prompt = """
    당신은 이미지를 요약하여 검색을 위해 사용할 수 있도록 돕는 어시스턴트입니다.
    이 요약은 임베딩되어 원본 이미지를 검색하는 데 사용됩니다.
    이미지 검색에 최적화된 간결한 요약을 작성하세요.
    """
    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_base64}"
                        },
                    },
                ]
            )
        ]
    )
    return msg.content


In [23]:
# 이미지 요약을 저장하는 리스트
image_summaries = []

for img_base64 in img_base64_list:
    image_summary = image_summarize(img_base64)     # 이미지 요약
    image_summaries.append(image_summary)           # 요약된 이미지 저장

In [32]:
image_summaries[0]

'2024년 말라리아 주간소식지 (42주차) (10.13.-10.19.) - KDCA (Korea Disease Control and Prevention Agency)'

In [35]:
# from langchain.retrievers import MultiVectorRetriever
# from langchain_community.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.stores import InMemoryStore


from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3"
    )

# 분할한 텍스트들을 색인할 벡터 저장소
vectorstore = Chroma(collection_name="multi_modal_rag",
                     embedding_function=embedding)

# 원본문서 저장을 위한 저장소 선언
docstore = InMemoryStore()
id_key = "doc_id"

# # 검색기
# retriever = MultiVectorRetriever(
#     vectorstore=vectorstore,
#     docstore=docstore,
#     id_key=id_key,
# )

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
import uuid

# 원본 텍스트 데이터 저장
doc_ids = [str(uuid.uuid4()) for _ in texts]
retriever.docstore.mset(list(zip(doc_ids, texts)))

# 원본 테이블 데이터 저장
table_ids = [str(uuid.uuid4()) for _ in tables]
retriever.docstore.mset(list(zip(table_ids, tables)))

# 원본 이미지(base64) 데이터 저장
img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

In [ ]:
from langchain.schema.document import Document

# 텍스트 요약 벡터 저장
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)

# 테이블 요약 벡터 저장
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)

# 이미지 요약 벡터 저장

summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)

In [ ]:
docs = retriever.invoke(
    "말라리아 군집 사례는 어떤가요? "
)

In [ ]:
len(docs)

In [ ]:
from base64 import b64decode

def split_image_text_types(docs):
    # 이미지와 텍스트 데이터를 분리
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {
        "images": b64,
        "texts": text
    }

docs_by_type = split_image_text_types(docs)

In [ ]:
len(docs_by_type["images"])

In [ ]:

len(docs_by_type["texts"])

In [ ]:
docs_by_type["images"][0][:100]

In [ ]:
docs_by_type["texts"]

In [ ]:
from IPython.display import display, HTML

def plt_img_base64(img_base64):
    # base64 이미지로 html 태그를 작성합니다
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # html 태그를 기반으로 이미지를 표기합니다
    display(HTML(image_html))

plt_img_base64(docs_by_type["images"][0])

In [ ]:
docs_by_type["texts"][0]

In [ ]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def prompt_func(dict):
    format_texts = "\n".join(dict["context"]["texts"])
    text = f"""
    다음 문맥에만 기반하여 질문에 답하세요. 문맥에는 텍스트, 표, 그리고 아래 이미지가 포함될 수 있습니다:
    질문: {dict["question"]}

    텍스트와 표:
    {format_texts}
    """

    prompt = [
        HumanMessage(
            content=[
                {"type": "text", "text": text},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{dict['context']['images'][0]}"}},
            ]
        )
    ]

    return prompt


model = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=1024)

# RAG 파이프라인
chain = (
        {"context": retriever | RunnableLambda(split_image_text_types), "question": RunnablePassthrough()}
        | RunnableLambda(prompt_func)
        | model
        | StrOutputParser()
)

In [ ]:
chain.invoke(
    "말라리아 군집 사례는 어떤가요?"
)